In [1]:
import os
import sys
import time
import numpy as np
import math
import json
from tqdm import tqdm_notebook as tqdm
import pathlib
from pathlib import Path
import time
import skimage.measure


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import (NNConv, GMMConv, GraphConv, Set2Set)
from torch_geometric.nn import (SplineConv, graclus, max_pool, max_pool_x, global_mean_pool)

import plyfile
from plyfile import PlyData
import trimesh
#from neuralnet_pytorch.metrics import chamfer_loss

import trimesh

#from visualization_utils import plot_mesh_3d

import deep_sdf
import deep_sdf.workspace as ws
#from models import *
#from datasets import *
#from custom_utils import *

from sklearn.neighbors import KDTree
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from pyntcloud import PyntCloud

import matplotlib.pyplot as plt 
%matplotlib inline


%load_ext autoreload
%autoreload 2
%matplotlib widget



In [3]:
def computeAvgTransform():
    objects = list()
    for (dirpath, dirnames, filenames) in os.walk("/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/transforms/"):
        objects += [os.path.join(dirpath, file) for file in filenames if file[-4:] == '.npy']
    
    matricies = []
    for obj in objects:
        matricies.append(np.load(obj))
    
    return np.mean(np.array(matricies), axis=0)

def transformPoints(points, matrix):
    matrix = torch.cuda.FloatTensor(matrix)
    column = torch.zeros((len(points), 1), device="cuda:0") + 1
    stacked = torch.cat([points, column], dim=1)
    transformed = torch.matmul(matrix, stacked.T).T[:, :3]
    return transformed

def make_mesh_from_points(points, ply_mesh):
    transformed_points = transformPoints(points, AvgTransform)
    
    edges = trimesh.geometry.faces_to_edges(ply_mesh['face']['vertex_indices'])
    np_points = transformed_points.cpu().detach().numpy()
    edge_attr = [np_points[a] - np_points[b] for a, b in edges]

    data = torch_geometric.data.Data(x   = transformed_points, 
                                     pos = transformed_points, 
                                     face = torch.tensor(ply_mesh['face']['vertex_indices'], 
                                                         dtype=torch.long).to('cuda:0').t(),
                                     edge_attr=torch.tensor(edge_attr, dtype=torch.float).to('cuda:0'),
                                     edge_index=torch.tensor(edges, dtype=torch.long).t().contiguous().to('cuda:0'))
    return data


def compute_volume(vertices_in, faces_in):

    bs, nv = vertices_in.shape[:2]
    bs, nf = faces_in.shape[:2]
    device = vertices_in.device
    faces = faces_in + (torch.arange(bs, dtype=torch.int32).to(device) * nv)[:, None, None]
    vertices = vertices_in #.reshape((bs * nv / 2, 3))
    # pytorch only supports long and byte tensors for indexing
    face_coordinates = vertices[faces.long()].squeeze(0)

    x1y2z3 = face_coordinates[:,0,0]*face_coordinates[:,1,1]*face_coordinates[:,2,2]
    x2y1z3 = face_coordinates[:,1,0]*face_coordinates[:,0,1]*face_coordinates[:,2,2]
    x3y1z2 = face_coordinates[:,2,0]*face_coordinates[:,0,1]*face_coordinates[:,1,2]
    x1y3z2 = face_coordinates[:,0,0]*face_coordinates[:,2,1]*face_coordinates[:,1,2]
    x2y3z1 = face_coordinates[:,1,0]*face_coordinates[:,2,1]*face_coordinates[:,0,2]
    x3y2z1 = face_coordinates[:,2,0]*face_coordinates[:,1,1]*face_coordinates[:,0,2]

    V = x1y2z3 - x2y1z3 + x3y1z2 - x1y3z2 + x2y3z1 - x3y2z1
    return 1.0/6.0*torch.abs(torch.sum(V))

def compute_lift_faces_diff(data_instance, answers, axis=0):
    pressures = torch.mean(answers[data_instance.face, 0], axis=0)

    # TODO: cahnge to x if needed
    pos = data_instance.x
    cross_prod = (pos[data_instance.face[1]] - pos[data_instance.face[0]]).cross(
                  pos[data_instance.face[2]] - pos[data_instance.face[0]])
    mult = -cross_prod[:, axis] / 2
    lift = torch.mul(pressures, mult)
    return torch.sum(lift[~torch.isnan(lift)])

def boundsLoss(points, box=[(-1, 1, 0)]):
    loss = 0
    for l, r, i in box:
        loss +=  torch.mean(F.relu(-points[:, i] + l))  \
               + torch.mean(F.relu( points[:, i] - r))
    return loss

def innerBoundsLoss(points, r=1, center=(0, 0, 0)):
    radiuses = torch.sum( (points - torch.Tensor(center).to('cuda:0')) ** 2 , dim=1)
    return torch.mean(F.relu(r - radiuses))

def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.05, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    

    return loss
def get_trimesh_from_torch_geo_with_colors(mesh, preds, vmin=-8, vmax=8):
    norm = mpl.colors.Normalize(vmin= vmin, vmax=vmax)
    cmap = cm.hot
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    
    verticies = mesh.x.cpu().detach()
    faces = mesh.face.t().cpu().detach()
    return trimesh.Trimesh(vertices=verticies, faces=faces, 
                           vertex_colors=list(map(lambda c: m.to_rgba(c),  preds[:, 0].cpu().detach())))
   

In [4]:
class SplineBlock(nn.Module):
    def __init__(self, num_in_features, num_outp_features, mid_features, kernel=3, dim=3, batchnorm1=True):
        super(SplineBlock, self).__init__()
        self.batchnorm1 = batchnorm1
        self.conv1 = SplineConv(num_in_features, mid_features, dim, kernel, is_open_spline=False)
        if self.batchnorm1:
            self.batchnorm1 = torch.nn.BatchNorm1d(mid_features)
        self.conv2 = SplineConv(mid_features, 2 * mid_features, dim, kernel, is_open_spline=False)
        self.batchnorm2 = torch.nn.BatchNorm1d(2 * mid_features)
        self.conv3 = SplineConv(2 * mid_features + 3, num_outp_features, dim, kernel, is_open_spline=False)
  
    def forward(self, res, data):
        if self.batchnorm1:
            res = F.elu(self.batchnorm1(self.conv1(res, data.edge_index, data.edge_attr)))
        else:
            res = F.elu(self.conv1(res, data.edge_index, data.edge_attr))
        res = F.elu(self.batchnorm2(self.conv2(res, data.edge_index, data.edge_attr)))
#         res = F.elu(self.conv2(res, data.edge_index, data.edge_attr))
        res = torch.cat([res, data.pos], dim=1)
        res = self.conv3(res, data.edge_index, data.edge_attr)
        return res

class SplineCNN8Residuals(nn.Module):
    def __init__(self, num_features, kernel=3, dim=3):
        super(SplineCNN8Residuals, self).__init__()
        self.block1 = SplineBlock(num_features, 16, 8, kernel, dim)
        self.block2 = SplineBlock(16, 64, 32, kernel, dim)
        self.block3 = SplineBlock(64, 64, 128, kernel, dim)
        self.block4 = SplineBlock(64, 8, 16, kernel, dim)
        self.block5 = SplineBlock(11, 32, 16, kernel, dim)
        self.block6 = SplineBlock(32, 64, 32, kernel, dim)
        self.block7 = SplineBlock(64, 64, 128, kernel, dim)
        self.block8 = SplineBlock(75, 4, 16, kernel, dim)

    def forward(self, data):
        res = data.x
        res = self.block1(res, data)
        res = self.block2(res, data)
        res = self.block3(res, data)
        res4 = self.block4(res, data)
        res = torch.cat([res4, data.x], dim=1)
        res = self.block5(res, data)
        res = self.block6(res, data)
        res = self.block7(res, data)
        res = torch.cat([res, res4, data.x], dim=1)
        res = self.block8(res, data)
        return res


In [5]:
def load_model(experiment_directory, checkpoint):
    specs_filename = os.path.join(experiment_directory, "specs.json")

    if not os.path.isfile(specs_filename):
        raise Exception(
            'The experiment directory does not include specifications file "specs.json"'
        )

    specs = json.load(open(specs_filename))

    arch = __import__("networks." + specs["NetworkArch"], fromlist=["Decoder"])

    latent_size = specs["CodeLength"]

    decoder = arch.Decoder(latent_size, **specs["NetworkSpecs"])

    decoder = torch.nn.DataParallel(decoder)

    saved_model_state = torch.load(
        os.path.join(experiment_directory, ws.model_params_subdir, checkpoint + ".pth")
    )
    saved_model_epoch = saved_model_state["epoch"]

    decoder.load_state_dict(saved_model_state["model_state_dict"])

    decoder = decoder.module.cuda()

    decoder.eval()
    
    return decoder
def create_mesh(
    decoder, latent_vec, filename='', N=256, max_batch=32 ** 3, offset=None, scale=None
):
    start = time.time()
    ply_filename = filename

    decoder.eval()

    # NOTE: the voxel_origin is actually the (bottom, left, down) corner, not the middle
    voxel_origin = [-1, -1, -1]
    voxel_size = 2.0 / (N - 1)

    overall_index = torch.arange(0, N ** 3, 1, out=torch.LongTensor())
    samples = torch.zeros(N ** 3, 4)

    # transform first 3 columns
    # to be the x, y, z index
    samples[:, 2] = overall_index % N
    samples[:, 1] = (overall_index.long() / N) % N
    samples[:, 0] = ((overall_index.long() / N) / N) % N

    # transform first 3 columns
    # to be the x, y, z coordinate
    samples[:, 0] = (samples[:, 0] * voxel_size) + voxel_origin[2]
    samples[:, 1] = (samples[:, 1] * voxel_size) + voxel_origin[1]
    samples[:, 2] = (samples[:, 2] * voxel_size) + voxel_origin[0]

    num_samples = N ** 3

    samples.requires_grad = False

    head = 0

    while head < num_samples:
        sample_subset = samples[head : min(head + max_batch, num_samples), 0:3].cuda()

        samples[head : min(head + max_batch, num_samples), 3] = \
                decode_sdf(decoder, latent_vec, sample_subset).squeeze(1).detach().cpu()
        head += max_batch

    sdf_values = samples[:, 3]
    sdf_values = sdf_values.reshape(N, N, N)

    end = time.time()
    #print("sampling takes: %f" % (end - start))

    return convert_sdf_samples_to_ply(
        sdf_values.data.cpu(),
        voxel_origin,
        voxel_size,
        ply_filename + ".ply",
        offset,
        scale,
    )

def convert_sdf_samples_to_ply(
    pytorch_3d_sdf_tensor,
    voxel_grid_origin,
    voxel_size,
    ply_filename_out,
    offset=None,
    scale=None,
):
    """
    Convert sdf samples to .ply

    :param pytorch_3d_sdf_tensor: a torch.FloatTensor of shape (n,n,n)
    :voxel_grid_origin: a list of three floats: the bottom, left, down origin of the voxel grid
    :voxel_size: float, the size of the voxels
    :ply_filename_out: string, path of the filename to save to

    This function adapted from: https://github.com/RobotLocomotion/spartan
    """
    start_time = time.time()

    numpy_3d_sdf_tensor = pytorch_3d_sdf_tensor.numpy()

    verts, faces, normals, values = skimage.measure.marching_cubes_lewiner(
        numpy_3d_sdf_tensor, level=0.0, spacing=[voxel_size] * 3
    )

    # transform from voxel coordinates to camera coordinates
    # note x and y are flipped in the output of marching_cubes
    mesh_points = np.zeros_like(verts)
    mesh_points[:, 0] = voxel_grid_origin[0] + verts[:, 0]
    mesh_points[:, 1] = voxel_grid_origin[1] + verts[:, 1]
    mesh_points[:, 2] = voxel_grid_origin[2] + verts[:, 2]

    # apply additional offset and scale
    if scale is not None:
        mesh_points = mesh_points / scale
    if offset is not None:
        mesh_points = mesh_points - offset

    # try writing to the ply file

    num_verts = verts.shape[0]
    num_faces = faces.shape[0]

    verts_tuple = np.zeros((num_verts,), dtype=[("x", "f4"), ("y", "f4"), ("z", "f4")])
    norms_tuple = np.zeros((num_verts,), dtype=[("x", "f4"), ("y", "f4"), ("z", "f4")])

    for i in range(0, num_verts):
        verts_tuple[i] = tuple(mesh_points[i, :])
        norms_tuple[i] = tuple(normals[i, :])

    faces_building = []
    for i in range(0, num_faces):
        faces_building.append(((faces[i, :].tolist(),)))
    faces_tuple = np.array(faces_building, dtype=[("vertex_indices", "i4", (3,))])

    el_verts = plyfile.PlyElement.describe(verts_tuple, "vertex")
    el_faces = plyfile.PlyElement.describe(faces_tuple, "face")
    el_norms = plyfile.PlyElement.describe(norms_tuple, "normals")

    ply_data = plyfile.PlyData([el_verts, el_faces, el_norms])
    return ply_data

def decode_sdf(decoder, latent_vector, queries):
    num_samples = queries.shape[0]

    if latent_vector is None:
        inputs = queries
    else:
        latent_repeat = latent_vector.expand(num_samples, -1)
        inputs = torch.cat([latent_repeat, queries], 1)

    sdf = decoder(inputs)

    return sdf

def plot_points_from_torch(tensor):
    cloud = PyntCloud(pd.DataFrame(tensor, columns=['x', 'y', 'z']))
    cloud.plot(background='white', initial_point_size=0.003)

In [24]:
!pwd

/home/qiu/Non-convex


In [6]:
# data_instance= make_data_instance_from_stl(
#                '/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0003_0015.fld') # /cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0002_0005.fld
device = "cuda:0"

# modelOld = SplineCNN8(3, batchnorm1=False)
# modelOld.load_state_dict(torch.load("Expirements/SplineCNN8.nn"))
# modelOld = modelOld.to(device)

model = SplineCNN8Residuals(3)
model.load_state_dict(torch.load("/cvlabdata2/home/artem/DeepSDF/Expirements/Networks15/normilized_full_latest.nn"))
model = model.to(device)
model = model.eval()

In [7]:
experiment_directory = "/cvlabdata2/home/artem/DeepSDF/examples/cars_cleared/"
checkpoint = "latest"

AvgTransform = computeAvgTransform()
decoder = load_model(experiment_directory, checkpoint).to('cuda:0')
latent_vectors = ws.load_latent_vectors(experiment_directory, checkpoint)
LATENT_KD_TREE = KDTree(np.array([lv.cpu().detach().numpy()[0] for lv in latent_vectors]))
latent_size = latent_vectors[0].size()[1]
print(f"{len(latent_vectors)} of latent vectors, each {latent_size} long")

1205 of latent vectors, each 256 long


# Vertex-vise Optimization

# BB Optimization

# Free Foam Optimization

# Free Foam Furrier Optimization

# AtlasNet Optimization

# DeepSDF Optimization

In [8]:
def method4_to_arbitatry_loss(points, ply_mesh, model, constraint_rad=0.1, axis=0):

    initial_dir = points.grad.clone()
    points.grad.data.zero_()

    mesh = make_mesh_from_points(points, ply_mesh)
    #signs = compute_signs_for_loss(mesh, transformPoints(normals, AvgTransform))
    local_preds = model(mesh)
    loss = calculate_loss(mesh, local_preds, axis=axis, constraint_rad=constraint_rad)
    loss.backward()

    sign = [-p1.dot(p2) for p1, p2 in zip(initial_dir, points.grad)]
    
    return sign, loss, local_preds, mesh

def optimize_shape_deepSDF(decoder, latent, ref_latent, initial_points=None, num_points=None, 
                           num_iters=100, point_iters=100,  punch_lr_at_reindex_by=1, num_neignours_constr=10,
                           reindex_latent_each=50, reindex_num_iterations=500, reindex_num_samples=100,
                           lr=0.2, decreased_by=2, adjust_lr_every=10, alpha_penalty=0.05,
                           multiplier_func=method4_to_arbitatry_loss, verbose=None, save_to_dir=None, N=256):

    def adjust_learning_rate(
        initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every
    ):
        lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every)) \
                        * ((punch_lr_at_reindex_by) ** (num_iterations // reindex_latent_each))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        return lr
    
    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))

    decoder.eval()
    latent = latent.clone()
    latent.requires_grad = True
    optimizer = torch.optim.SGD([latent], lr=lr)

    loss_plot = []
    latent_dist = []
    lr_plot = []

    #if initial_points is not None:
    #    points = initial_points.clone()
    #else:
    #    points = get_points_from_latent(decoder, latent, N=N, point_num=num_points)

    for i in range(num_iters):

        time_start = time.time()

        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")

#             if i > 0 and i == reindex_latent_each:
#                 new_latent = get_latent_from_mesh(decoder, latent_size=latent.size()[1], 
#                                                   num_iterations=reindex_num_iterations, 
#                                                   num_samples=reindex_num_samples)
#                 latent = torch.Tensor(new_latent.cpu().detach().numpy()).cuda()
#                 latent.requires_grad = True
#                 optimizer = torch.optim.Adam([latent], lr=lr)

        cur_rl = adjust_learning_rate(lr, optimizer, i, decreased_by, adjust_lr_every)

        with torch.no_grad():
            ply_mesh = create_mesh( decoder,
                                    latent,
                                    N=N,
                                    max_batch=int(2 ** 18),
                                    offset=None,
                                    scale=None)

        points = torch.cuda.FloatTensor(np.hstack(( ply_mesh['vertex']['x'][:, None], 
                                                    ply_mesh['vertex']['y'][:, None], 
                                                    ply_mesh['vertex']['z'][:, None])))

        points.requires_grad = True

        sdf_value = deep_sdf.utils.decode_sdf(decoder, latent, points)
        sdf_value.backward(torch.ones([len(points), 1], dtype=torch.float32).cuda())

        mults, loss_value, preds, transformed_mesh = multiplier_func(points, ply_mesh)         
        multipliers = torch.cuda.FloatTensor(mults)

        optimizer.zero_grad()
        sdf_value = torch.squeeze(deep_sdf.utils.decode_sdf(decoder, latent, points))

        final_loss = torch.sum(sdf_value * multipliers)
        final_loss.backward()
#             print( "Latent size: ", torch.sum(latent ** 2) )
#             print( "Latent grad: ", torch.sum(latent.grad ** 2) )

#           sdf_value.backward(multipliers)


        # Soft-constraints
        distances, indeces = LATENT_KD_TREE.query(latent.cpu().detach(), k=num_neignours_constr)
        penalty = torch.mean(
                    torch.stack([torch.sum( 
                                    (latent - latent_vectors[indeces[0][i]]) ** 2
                                 )
                                 for i in range(len(indeces[0]))]
                               )
                    )
        apenalty = penalty * alpha_penalty
        apenalty.backward()
        #print("Latent grad penalized: ", torch.sum(latent.grad ** 2))

        optimizer.step()

        tri_mesh = get_trimesh_from_torch_geo_with_colors(transformed_mesh, preds)
        tri_mesh.export(save_path)
        np.save(preds_save_path, preds.cpu().detach().numpy())

        #if save_to_dir is not None:
        #    plot_points_from_torch

        loss_plot.append(loss_value.cpu().detach().numpy())
        latent_dist.append(torch.sum((latent - ref_latent) ** 2 ).cpu().detach().numpy() )
        lr_plot.append(penalty.cpu().detach().numpy())

        time_end = time.time()

        if verbose is not None and i % verbose == 0:
            print('Iter ', i, 'Loss: ', loss_value.detach().cpu().numpy(), ' LD: ', lr_plot[-1])
        np.save(os.path.join(save_to_dir, "latent%d.npy" % i), latent.cpu().detach().numpy())
        
    np.save(os.path.join(save_to_dir, "loss_plot.npy"), loss_plot)
    np.save(os.path.join(save_to_dir, "latent_dist.npy"), latent_dist)
    np.save(os.path.join(save_to_dir, "lr_plot.npy"), lr_plot)



def make_full_transformation(initial_latent, ref_latent, experiment_name, 
                             decoder, model, alpha_penalty=0.05, constraint_rad=0.1, axis=0, **kwargs):
    '''
    kwargs:
        num_iters=1000, 
        adjust_lr_every=10, 
        decreased_by=1.2,
        lr=0.005,
        
        reindex_latent_each=100,
        punch_lr_at_reindex_by=1,
        reindex_num_iterations=500, 
        reindex_num_samples=100,
        
        verbose=10,
    '''

    #ref_points = get_points_from_latent(decoder, ref_latent, N=128)

    save_to_dir = experiment_name
    if not os.path.exists(save_to_dir):
        os.makedirs(save_to_dir)

    #np.save(os.path.join(save_to_dir, "target_verts.npy"), ref_points)

    optimize_shape_deepSDF(decoder, initial_latent, ref_latent, initial_points=None,
                                           alpha_penalty=alpha_penalty,
                                           num_points=None, point_iters=2,
                                           multiplier_func=lambda x, y: 
                                               method4_to_arbitatry_loss(x, y, model, 
                                                                         constraint_rad=constraint_rad, 
                                                                         axis=axis),
                                           save_to_dir=save_to_dir, **kwargs)
    

In [ ]:
np.random.seed(102)
LATENT_TO_OPTIMIZE = latent_vectors[32]
make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Experiments/OptimizationPaper/', decoder, model,
                         alpha_penalty=0.2, axis=0,
                         constraint_rad=0.05,
                         num_iters=5,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=1,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

Iter  0 Loss:  0.09731689  LD:  0.3987185
Iter  1 Loss:  0.0863896  LD:  0.36342707
Iter  2 Loss:  0.090333864  LD:  0.34481293


In [ ]:

Iter  0 Loss:  0.09731689  LD:  0.3987185
Iter  1 Loss:  0.08640832  LD:  0.36342707
Iter  2 Loss:  0.09052619  LD:  0.34481156

In [8]:
preds = np.load("Expirements/OptimizationPaper/predictions/00000.npy")
mesh = trimesh.load_mesh("Expirements/OptimizationPaper/meshes/00000.ply")
loss_plot = np.load("Expirements/OptimizationPaper/loss_plot.npy", allow_pickle=True)
lr_plot = np.load("Expirements/OptimizationPaper/lr_plot.npy", allow_pickle=True)
latent0 = np.load("Expirements/OptimizationPaper/latent0.npy", allow_pickle=True)
latent1 = np.load("Expirements/OptimizationPaper/latent1.npy", allow_pickle=True)


In [10]:
preds.mean(axis=0)

array([-0.13065428, -0.01960552, -0.06631227,  0.02476818], dtype=float32)

In [16]:
loss_plot

array([0.09731689, 0.0864084 ], dtype=float32)

In [27]:
LATENT_TO_OPTIMIZE = latent_vectors[32]

In [31]:
np.linalg.norm(LATENT_TO_OPTIMIZE.detach().cpu().numpy() - latent1)**2

0.028872395351264624

In [20]:
latent_dist = np.load("Expirements/OptimizationPaper/latent_dist.npy")
latent_dist

array([0.01667565, 0.0288724 ], dtype=float32)

In [17]:
lr_plot

array([tensor(0.3987, device='cuda:0', requires_grad=True),
       tensor(0.3634, device='cuda:0', requires_grad=True)], dtype=object)

In [32]:
mesh2 =  trimesh.load_mesh("Expirements/OptimizationPaper/meshes/00001.ply")
#mesh2.show()

In [33]:
#mesh.show()

## Run Everything

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.15, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/cabinBackSmall15', decoder, model,
                         alpha_penalty=0.2, axis=0,
                         constraint_rad=0.05,
                         num_iters=30,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=None,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

# Check Simulation

In [ ]:
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/ListenerCheck/processed/fld/0000_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/ListenerCheck/processed/fld/0001_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/ListenerCheck/processed/fld/0002_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
plt.show()

In [ ]:
di0 = make_data_instance_from_stl("../Expirements/SimulatedResults/foam_npy/fld/0000_0005.fld")
di1 = make_data_instance_from_stl("../Expirements/SimulatedResults/foam_npy/fld/0001_0005.fld")

In [ ]:
plt.hist(di0.y[:, 0], bins=100)
plt.show()

In [ ]:
plt.hist(di1.y[:, 0], bins=100)
plt.show()

In [ ]:
mesh = trimesh.load('/cvlabdata2/home/artem/Data/cars_refined/simulated/stl/0005.stl')

In [ ]:
mesh.show()

In [ ]:
path = '../Expirements/SavedTransforms/DeepSDF-CFD8-m3-holes/meshes/00000.ply'
tri_mesh = trimesh.load(path)
points = torch.tensor(tri_mesh.vertices, dtype=torch.float).to("cuda:0")

In [ ]:
innerBoundsLoss(points, r=0.12**2, center=(0.55, 0, 0))

In [ ]:
from numpy import genfromtxt
bvecs = genfromtxt('examples/cars/vine_latentVecs_tll_0.1_1.csv', delimiter=' ', skip_header=1)

In [ ]:
bvecs = [torch.FloatTensor(v).to('cuda:0') for v in bvecs]

In [ ]:
ply_mesh = create_mesh( decoder,
                        latent_vectors[0],
                        N=128,
                        max_batch=int(2 ** 18),
                        offset=None,
                        scale=None)

points = torch.cuda.FloatTensor(
    np.array([ply_mesh['vertex']['x'], ply_mesh['vertex']['y'], ply_mesh['vertex']['z']]).transpose()
)

transformed_points = make_mesh_from_points(points, ply_mesh)

In [ ]:
pred = model(transformed_points)

In [ ]:
mesh = get_trimesh_from_torch_geo_with_colors(transformed_points, pred.cpu().detach())

In [ ]:
# mesh.show()

In [ ]:
n = mesh.export('examples/cars/BeautifiedMeshes/o0.ply')

In [ ]:
get_latent_from_mesh_cpu()

In [ ]:
edges = trimesh.geometry.faces_to_edges(ply_mesh['face']['vertex_indices'])
edge_attr = torch.stack([points[a] - points[b] for a, b in edges])

In [ ]:
print(edge_attr[:10])

In [ ]:
cloud = PyntCloud(pd.DataFrame(points.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
cloud = PyntCloud(pd.DataFrame(points_transformed.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
di = make_data_instance_from_stl('/cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0007_0005.fld')

In [ ]:
cm = get_trimesh_from_torch_geo_with_colors(di, di.y)

In [ ]:
preds = model()

In [ ]:
cm.show()

In [ ]:
transformed_points

In [ ]:
cloud = PyntCloud(pd.DataFrame(di.pos.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
cloud = PyntCloud(pd.DataFrame(transformed_points.pos.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
with torch.no_grad():
    ply_mesh = create_mesh( decoder,
                            latent,
                            N=128,
                            max_batch=int(2 ** 18),
                            offset=None,
                            scale=None)

points = torch.cuda.FloatTensor(np.hstack(( ply_mesh['vertex']['x'][:, None], 
                                            ply_mesh['vertex']['y'][:, None], 
                                            ply_mesh['vertex']['z'][:, None])))

mesh = trimesh.Trimesh(vertices=points.cpu().detach(), faces=ply_mesh['face']['vertex_indices'])
edge_attr = [mesh.vertices[a] - mesh.vertices[b] for a, b in mesh.edges]

data = torch_geometric.data.Data(x  = points, 
                                 pos= torch.tensor(mesh.vertices, dtype=torch.float).to('cuda:0'), 
                                 faces = torch.tensor(mesh.faces, dtype=torch.long).to('cuda:0'),
                                 edge_attr = torch.tensor(edge_attr, dtype=torch.float).to('cuda:0'),
                                 edge_index= torch.tensor(mesh.edges, dtype=torch.long).t().contiguous().to('cuda:0'))

In [ ]:
data_instance= make_data_instance_from_stl(
                    '/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/0100_0005.fld', data_step=1)
mesh = trimesh.Trimesh(vertices=data_instance.pos, faces=data_instance.faces)
a = mesh.export('../Expirements/data/original_mesh.ply')

# Expirements with PreprocessMesh

In [ ]:
mesh.show()

In [ ]:
latent = get_latent_from_mesh_cpu(decoder, latent_size, mesh, num_iterations=300, num_samples=200000)

In [ ]:
latent_gpu = get_latent_from_mesh(decoder, latent_size, mesh_path='../Expirements/data/original_mesh.ply', 
                         num_iterations=300, num_samples=200000)

In [ ]:
gpu_points = deep_sdf.data.read_sdf_samples_into_ram('../Expirements/data/original_SDF.npz')
gpu_points[1][:, 3].max()

In [ ]:
cloud = PyntCloud(pd.DataFrame(np.array(gpu_points[0][:, :3]), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.03, elev=-45, azim=45)

In [ ]:
points, sdf = sample_sdf_near_surface(mesh)
sdfs = np.hstack((points, sdf[:, None]))
data_sdf = [torch.from_numpy(sdfs[sdfs[:, 3] > 0, :]), 
            torch.from_numpy(sdfs[sdfs[:, 3] < 0, :])]

In [ ]:
print(data_sdf[0].shape)
print(gpu_points[0].shape)

In [ ]:
cloud = PyntCloud(pd.DataFrame(np.array(data_sdf[0][:, :3]), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.03, elev=-45, azim=45)

In [ ]:
plot_mesh_from_vector(decoder, latent, N=128)

In [ ]:
plydata = create_mesh(decoder, latent_gpu)

In [ ]:
plydata.write("../Expirements/mesh.ply")